# Building my Reddit Scraper

I have annotated my webscraper below, and commented out the cells that interact with the actual CSV that contains the data, since the python script that collects my data edits and rewrites that CSV. 

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime

In [13]:
#setting up the driver

ser = Service('../chromedriver/chromedriver.exe')
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)
driver.get('https://www.reddit.com/')

'Without saying the name, what is a TV show that is instantly recognizable based on one quote?\n\n'

In [ ]:
#getting the previously scraped data
# old_data = pd.read_csv('../data/reddit_data.csv')

In [ ]:
#grabbing the html for each post
post = driver.find_element_by_tag_name('body')

In [ ]:
#manually scrolling reddit using page down 500 times
count = 0
while count < 500:
    post.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)
    count += 1

In [ ]:
#Get the html from the scrolled page and create a soup    
html = driver.page_source

soup = BeautifulSoup(html)

In [ ]:
#grabbing all posts
all_posts = soup.findAll('div',{'class':'_1oQyIsiPHYt6nx7VOmd1sz'})

In [ ]:
#getting the current time to be saved with each scrape
current_time = datetime.datetime.now()

In [ ]:
#creating a dictionary of each individual post
post_list = []
for post in all_posts:
 
    post_dict = {}
    try:    
        #if this first line of code throws an error, post does not contain
        #image, goes on to next block
        post.find('img',{'alt':'Post image'}).get('src')
        #append the relevant information to our list of dictionaries
        post_list.append({
        #get the unique post id
        'id':post.get('id'),
        #get the post title
        'title':post.find('h3', {'class':'_eYtD2XCVieq6emjKBH3m'}).text,
        #get the number of comments
        'comments':post.find("span", {'class': 'FHCV02u6Cp2zYL0fhQPsO'}).text,
        #find which subreddit post was posted to
        'subreddit':(post.find('a', {
            'class':'_3ryJoIoycVkA88fy40qNJc'})).get('href'),
        #get how many hours ago it was posted
        'time':post.find('a',{'class':'_3jOxDPIQ0KaOWpzvSQo-1s'}).text,
        #get the number of upvotes
        'upvotes':post.find('div', {
            'class':'_1rZYMD_4xY3gRcSS3p8ODO _3a2ZHWaih05DgAOtvu6cIo'}).text,
        #note what time we scraped the data
        'scraped_time':current_time,
        #if the first line of code doesn't throw an error, post contains an
        #image so we mark it as such below
        'contains_image':'1',
        'contains_video':'0',
        'text_only':'0'
        })
    except:
        try:
            #if this line of code throws an error, then post also does not 
            #contain video, goes on to next block
            post.find('video',{'class':'_1EQJpXY7ExS04odI1YBBlj'}).get('src')
            #append the relevant information to our list of dictionaries
            post_list.append({
            'id':post.get('id'),
            'title':post.find('h3', {'class':'_eYtD2XCVieq6emjKBH3m'}).text,
            'comments':post.find("span", {
                'class': 'FHCV02u6Cp2zYL0fhQPsO'}).text,
            'subreddit':(post.find('a', {
                'class':'_3ryJoIoycVkA88fy40qNJc'})).get('href'),
            'time':post.find('a',{'class':'_3jOxDPIQ0KaOWpzvSQo-1s'}).text,
            'upvotes':post.find('div', {
                'class':'_1rZYMD_4xY3gRcSS3p8ODO _3a2ZHWaih05DgAOtvu6cIo'}).text,
            'scraped_time':current_time,
            'contains_image':'0',
            'contains_video':'1',
            'text_only':'0'
            })
        except:
            try:
                #if THIS block of code throws an error, then the post is an ad, 
                #live stream, or live poll
                #append the relevant information to our list of dictionaries
                post_list.append({
                'id':post.get('id'),
                'title':post.find('h3', {'class':'_eYtD2XCVieq6emjKBH3m'}).text,
                'comments':post.find("span", {
                    'class': 'FHCV02u6Cp2zYL0fhQPsO'}).text,
                'subreddit':(post.find('a', {
                    'class':'_3ryJoIoycVkA88fy40qNJc'})).get('href'),
                'time':post.find('a',{'class':'_3jOxDPIQ0KaOWpzvSQo-1s'}).text,
                'upvotes':post.find('div', {
                    'class':'_1rZYMD_4xY3gRcSS3p8ODO _3a2ZHWaih05DgAOtvu6cIo'}).text,
                'scraped_time':current_time,
                'contains_image':'0',
                'contains_video':'0',
                'text_only':'1'
                })
            except:
                pass


In [ ]:
#creating a dataframe from our list of dictionaries
new_data = pd.DataFrame(post_list)

#combining the new data with what we have scraped previously
combined_df = pd.concat([old_data, new_data])


#DO NOT RUN THIS SMALL BLOCK OF CODE OR IT WILL OVERWRITE THE DATA
# #saving df as csv
# combined_df.to_csv('../data/reddit_data.csv', index = False)



#closing the driver
driver.close()